In [ ]:
%matplotlib widget
# %matplotlib inline

In [ ]:
from pathlib import Path
import re
from itertools import product

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from fluidsim import load
from fluidsim.util import (get_dataframe_from_paths,
    times_start_last_from_path,
)


In [ ]:
height = 5.5
plt.rc("figure", figsize=(1.33 * height, height))

In [ ]:
path_base = Path("/fsnet/project/meige/2022/22STRATURBANIS/aniso")
path_base_occigen = path_base.parent / "from_occigen/aniso"
paths_all = sorted(list(path_base.glob(f"ns3d.strat*")) + list(path_base_occigen.glob(f"ns3d.strat*"))) 

def get_path_finer_resol(N, Rb):
    str_N = f"_N{N}"
    str_Rb = f"_Rb{Rb:.3g}"
    paths_couple = [p for p in paths_all if str_N in p.name and str_Rb in p.name]
    paths_couple.sort(key=lambda p: int(p.name.split("x")[1]), reverse=True)
    for path in paths_couple:
        t_start, t_last = times_start_last_from_path(path)
        if t_last > t_start + 1:    
            return path

        
        
def lprod(a, b):
    return list(product(a, b))


couples = (
    lprod([10, 20, 40], [5, 10, 20, 40, 80, 160])
    + lprod([30], [10, 20, 40])
    + lprod([6.5], [100, 200])
    + lprod([4], [250, 500])
    + lprod([3], [450, 900])
    + lprod([2], [1000, 2000])
    + lprod([0.66], [9000, 18000])
)
couples.remove((40, 160))


    
paths = []
for N, Rb in couples:
    paths.append(get_path_finer_resol(N, Rb))
        
[p.name for p in paths]

In [ ]:
def customize(result, sim):
    result["Rb"] = float(sim.params.short_name_type_run.split("_Rb")[-1])
    result["nx"] = sim.params.oper.nx 


df = get_dataframe_from_paths(paths, tmin="t_start+1", use_cache=True, customize=customize)

In [ ]:
df

In [ ]:
df[(df.Fh < 0.04) & (df.Fh > 0.02) & (df.R4 > 8)]

In [ ]:
def plot(df, x, y, logx=True, logy=False, c=None, vmin=None, vmax=None):
    ax = df.plot.scatter(
        x=x, y=y, logx=logx, logy=logy, c=c, edgecolors="k", vmin=vmin, vmax=vmax
    )
    pc = ax.collections[0]
    pc.set_cmap("viridis")
    plt.colorbar(pc, ax=ax)
    return ax

In [ ]:
tmp = df[df.R2 < 120]
ax = plot(tmp, "Fh", "R2", c=tmp.I_velocity)

In [ ]:
tmp = df[df.R2 < 120]
ax = plot(tmp, "Fh", "R2", c=tmp.I_dissipation)

In [ ]:
df["R2/Rb"] = df.R2/df.Rb
plot(df, "Rb", "R2/Rb", c=np.log10(df.Fh), vmin=-2, vmax=-0.8)

In [ ]:
df["tmp"] = 1/df.Fh
plot(df, "Fh", "tmp", c=np.log10(df.Rb), vmin=1, vmax=2)

In [ ]:
plot(df, "Fh", "R4", c=df.I_velocity, logy=True)

In [ ]:
plot(df, "Fh", "R2", c=df["epsK2/epsK"], logy=True, vmin=0, vmax=1);

In [ ]:
ax = plot(df, "Fh", "epsK2/epsK", c=np.log10(df["R2"]), vmin=0.5, vmax=2)
ax.set_ylim(bottom=0);

In [ ]:
df[df["epsK2/epsK"] < 0.6]

In [ ]:
plot(df, "Fh", "I_velocity", c=np.log10(df["R2"]), vmin=0.5, vmax=2)

In [ ]:
plot(df, "R2", "I_dissipation", c=np.log10(df["Fh"]), vmin=-2, vmax=-1)

In [ ]:
def plot2(dataframe, ax=None, color=None):
    return dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, ax=ax, color=color
    )


tab10 = plt.get_cmap("tab10")

Ns = sorted(df.N.unique())

ax = None
for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], ax=ax, color=tab10(iN))


fig = ax.figure

ax_sub = fig.add_axes([0.6, 0.6, 1.33 * 0.2, 0.2])


def plot2(dataframe, color=None):
    dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, logy=True, ax=ax_sub, color=color
    )


for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], color=tab10(iN))

ax_sub.set_xlabel("")
ax_sub.set_ylabel("")

xs = np.linspace(7e-2, 3.5e-1, 4)
ax_sub.plot(xs, 3e-2 * xs**-1)
ax_sub.text(0.1, 0.1, "$k^{-1}$")

xs = np.linspace(5e-1, 1.5, 4)
ax_sub.plot(xs, 3e-2 * xs**-2)
ax_sub.text(1, 0.05, "$k^{-2}$");

In [ ]:
plot(df, "Fh", "Gamma", c=np.log10(df["R2"]), vmin=0.5, vmax=2)

In [ ]:
plot(df, "Fh", "Gamma", c=np.log10(df["min_R"]), vmin=0.5, vmax=2)